<a href="https://colab.research.google.com/github/SarthakNarayan/DL-and-ML/blob/master/googlecolab/CharacterGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Using colors

In [0]:
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

print (color.GREEN + 'Hello World !' )
print("happens if u dont end it")
print(color.END)
print("will not happen now")

Hello World !
happens if u dont end it

will not happen now


##Imports Section

In [0]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision
import time

##Loading the data
Link to the repo containing text file<br/>
<https://github.com/udacity/deep-learning><br/>
Go to tensorboard and there you will find anna.txt

In [0]:
with open("/content/deep-learning/tensorboard/anna.txt" , 'r') as f:
    text = f.read()
print("Total Number of characters in the text {}".format(len(text)))
# characters_to_be_read = 500001
# text = text[:characters_to_be_read]

Total Number of characters in the text 1985223


##Tokenization
We want to map every character to a unique index since our network can only learn from numerical data.

In [0]:
# Separates every unique character present in the input file 
# chars is also known as vocabulary
chars = tuple(set(text))
print(chars)
print("No of unique characters {} \n".format(len(chars)))

# Gives each of the characters a unique number starting from 0 by creating a dictionary
map_int_to_char = dict(enumerate(chars))
print(map_int_to_char)

# Reverse map numbers to characters
map_char_to_int = {}
for ii , char in map_int_to_char.items():
    map_char_to_int[char]=ii
    
print(map_char_to_int , "\n")

# Since we have characters as numbers we need to encode the input text
encoded = []
for ch in text:
    encoded.append(map_char_to_int[ch])

print("Encoded Text")
print(encoded[:20])
print(len(encoded))

('`', 'Y', '1', 'v', 'U', '-', 'B', 'y', 'M', '2', 'o', ';', 'q', ' ', 'b', 'O', "'", 'l', 'g', '!', 'i', 'G', 'V', 'I', '7', '/', 'Z', '$', '5', 'P', 'K', '@', ':', 's', 'c', 'T', 'L', '%', 'C', 'h', 't', 'e', '\n', 'f', 'A', 'N', 'p', 'E', ',', 'H', 'j', '(', 'd', 'R', '?', 'm', 'u', 'n', '4', 'X', 'S', 'D', ')', 'w', '.', '&', 'x', '"', '6', 'r', '3', '8', '0', 'F', '_', 'W', '9', 'a', 'z', 'Q', 'k', 'J', '*')
No of unique characters 83 

{0: '`', 1: 'Y', 2: '1', 3: 'v', 4: 'U', 5: '-', 6: 'B', 7: 'y', 8: 'M', 9: '2', 10: 'o', 11: ';', 12: 'q', 13: ' ', 14: 'b', 15: 'O', 16: "'", 17: 'l', 18: 'g', 19: '!', 20: 'i', 21: 'G', 22: 'V', 23: 'I', 24: '7', 25: '/', 26: 'Z', 27: '$', 28: '5', 29: 'P', 30: 'K', 31: '@', 32: ':', 33: 's', 34: 'c', 35: 'T', 36: 'L', 37: '%', 38: 'C', 39: 'h', 40: 't', 41: 'e', 42: '\n', 43: 'f', 44: 'A', 45: 'N', 46: 'p', 47: 'E', 48: ',', 49: 'H', 50: 'j', 51: '(', 52: 'd', 53: 'R', 54: '?', 55: 'm', 56: 'u', 57: 'n', 58: '4', 59: 'X', 60: 'S', 61: 'D', 62: 

##Defining the variables

In [0]:
# Length of the sequence to be sampled from the text data
length_of_sequence = 1000
# no of batches required
batch_size = 10
# so step size will be length_of_sequence/batch_size

no_hidden_layer = 512
input_size = len(chars)
output_size = len(chars)
num_layers = 2
num_epochs = 20
print(input_size)

83


##One hot encoding
Now we need to convert our encoded array into one hot representation

In [0]:
# The comments are for understanding
# uncomment them to see how it happens
def one_hot(encoded_array , onehot_length):
    # for [[1,2,3]] shape will give (1,3)
    # size will give number of elements present
    one_hot = np.zeros((encoded_array.size , onehot_length) , dtype = np.float32)
#     print(one_hot.shape)
    
    # flatten the array
#     print(encoded_array.flatten())
#     print(np.arange(encoded_array.size))
    one_hot[np.arange(encoded_array.size), encoded_array.flatten()] = 1.0
#     print(one_hot)
    
    one_hot = one_hot.reshape((*encoded_array.shape, onehot_length))
    return one_hot

one_hot_encoded = one_hot(np.array([[1,2,3,4]]),6)
print(one_hot_encoded)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


##Creating Mini batches for training
we want our batches to be multiple sequences of some desired number of sequence steps.<br/>

In [0]:
def batcher(arr , batch_size , job='train'):
    length_of_array = len(arr)
    if job=='train':
        number_bathes = batch_size
    elif job=='generate':
        number_bathes = 1
        
    elements_per_batch = len(arr)//number_bathes
    total_elements = elements_per_batch*number_bathes
    arr = arr[:total_elements]
    arr = arr.reshape(number_bathes , -1)
    return arr

x = encoded[:100]
batched = batcher(np.array(x) , batch_size , 'train')
print(batched)
print(batched.shape)
print("\n Final shape of the input")
one_hot_encoded = one_hot(batched,len(chars))
print(one_hot_encoded.shape)

[[38 39 77 46 40 41 69 13  2 42]
 [42 42 49 77 46 46  7 13 43 77]
 [55 20 17 20 41 33 13 77 69 41]
 [13 77 17 17 13 77 17 20 80 41]
 [11 13 41  3 41 69  7 13 56 57]
 [39 77 46 46  7 13 43 77 55 20]
 [17  7 13 20 33 13 56 57 39 77]
 [46 46  7 13 20 57 13 20 40 33]
 [13 10 63 57 42 63 77  7 64 42]
 [42 47  3 41 69  7 40 39 20 57]]
(10, 10)

 Final shape of the input
(10, 10, 83)


##Model

In [0]:
class CharacterLstm(nn.Module):
    def __init__(self , no_hidden_layer , num_layers):
        super(CharacterLstm , self).__init__()
        self.no_hidden_layer = no_hidden_layer
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size , self.no_hidden_layer ,
                            self.num_layers , batch_first = True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(self.no_hidden_layer , output_size)
    
    def forward(self ,x , c0 , h0):
        lstm_out , (c , h) = self.lstm(x , (c0,h0))
        lstm_out = self.dropout(lstm_out)
        lstm_out = lstm_out.contiguous()
        out = lstm_out.view(-1,self.no_hidden_layer)
        out = self.fc(out)
        return out , c , h
    
charlstm = CharacterLstm(no_hidden_layer , num_layers)
charlstm.cuda()
print(charlstm)

CharacterLstm(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


##Training

In [0]:
optimizer = optim.Adam(charlstm.parameters() , lr=0.001)
criterion = nn.CrossEntropyLoss()
start_time = time.time()

for i in range(num_epochs):
    h0 = torch.zeros(num_layers , batch_size , no_hidden_layer).cuda()
    c0 = torch.zeros(num_layers , batch_size , no_hidden_layer).cuda()
    running_loss = 0
    charlstm.train()
    
    for counter,step in enumerate(range(0,len(encoded),length_of_sequence),1):
        optimizer.zero_grad()
        
        x = encoded[step:step+length_of_sequence]
        y = encoded[step+1:step+length_of_sequence+1]
        
        if(len(y)<length_of_sequence):
            print("Breaking")
            counter = counter-1
            break
        
        batched_x = batcher(np.array(x) , batch_size)
        batched_y = batcher(np.array(y) , batch_size)
        one_hot_x = one_hot(batched_x,len(chars))
        
        one_hot_x = torch.from_numpy(one_hot_x).cuda()
        batched_y = torch.from_numpy(batched_y).cuda()
        
        pred , c , h = charlstm(one_hot_x , c0 ,h0)
        c = c.data
        h = h.data
        batched_y = batched_y.reshape(pred.shape[0]).long()
        
        loss = criterion(pred , batched_y)
        loss.backward()
        
#         prevents exploding gradients problem in rnns and lstms
#         nn.utils.clip_grad_norm_(charlstm.parameters(), 5)
        optimizer.step()
        
        running_loss += loss
        
        if counter%2000==0:
            printing_loss = running_loss/counter
            print("Loss after {} steps in epoch {} is {}".format(counter,i+1,printing_loss))
            
        
        
        
    running_loss = running_loss/counter
    print(color.RED + "Loss after epoch {} is {}".format(i+1,running_loss) + color.END)
    
end_time = time.time()
print("Training Complete")
total_time = end_time - start_time
print("Total time taken {}".format(total_time))

Breaking
Loss after epoch 1 is 1.9476960897445679
Breaking
Loss after epoch 2 is 1.4227629899978638
Breaking
Loss after epoch 3 is 1.3010947704315186
Breaking
Loss after epoch 4 is 1.2352197170257568
Breaking
Loss after epoch 5 is 1.1909878253936768
Breaking
Loss after epoch 6 is 1.1578924655914307
Breaking
Loss after epoch 7 is 1.129645824432373
Breaking
Loss after epoch 8 is 1.1064649820327759
Breaking
Loss after epoch 9 is 1.0856143236160278
Breaking
Loss after epoch 10 is 1.067736029624939
Breaking
Loss after epoch 11 is 1.051243782043457
Breaking
Loss after epoch 12 is 1.0372923612594604
Breaking
Loss after epoch 13 is 1.024314045906067
Breaking
Loss after epoch 14 is 1.0127302408218384
Breaking
Loss after epoch 15 is 1.0028879642486572
Breaking
Loss after epoch 16 is 0.9923209547996521
Breaking
Loss after epoch 17 is 0.9837216734886169
Breaking
Loss after epoch 18 is 0.9756173491477966
Breaking
Loss after epoch 19 is 0.9668247103691101
Breaking
Loss after epoch 20 is 0.9602932929

##Text Generation

In [0]:
charlstm.eval()
def text_generation(input_sequence , length_to_generate = 100):
    
    h0 = torch.zeros(num_layers , 1 , no_hidden_layer).cuda()
    c0 = torch.zeros(num_layers , 1 , no_hidden_layer).cuda()
    x = input_sequence
    sentence = []
    
    for i in range(length_to_generate):
        x = [map_char_to_int[x]]
        batched_x = batcher(np.array(x) , 1 , 'generate')
        one_hot_x = one_hot(batched_x,len(chars))
        one_hot_x = torch.from_numpy(one_hot_x).cuda()
        pred , c , h = charlstm(one_hot_x , c0 ,h0)
        c = c.data
        h = h.data
        p = F.softmax(pred, dim=1).data.cpu()
        
        top_ch = np.arange(len(chars))
        top_k = 5
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        letter = map_int_to_char[char]

#         _ , letter = torch.max(p , dim=1)
#         letter = letter.item()
#         letter = map_int_to_char[letter]
        sentence.append(letter)
        x = letter
    
    return ''.join(sentence)

sentence = text_generation('b')
print(sentence)

e the the the the the the the the the the the the the the the the the the the the the the the the th
